<a href="https://colab.research.google.com/github/joyfulspace/ADP/blob/master/07_%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5_8_%EC%97%B0%EA%B4%80%EA%B7%9C%EC%B9%99%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 연관규칙분석
- 주어진 어떤 집합에서 원소들 간에 존재하는 규칙을 찾는 것
  (변수 간의 흥미로운 관계를 발견하기 위한 규칙기반 머신러닝)
- POS(Point Of Sale) 시스템에서 제품 간의 규칙성을 발견하는 기법
- 장바구니 분석, 생물 정보학, 웹 사용 마이닝 등에 활용
- 집합의 가능한 부분집합을 모두 구해야 해서 원소의 수가 많으면 연산량이 많아짐
  - 빈발 항목 집합: 부분 집합 중에서 빈번하게 나오는 아이템들의 집합
    - 예시: Apriori(선험 규칙)
  - 비빈발 항목 집합: 부분 집합 중에서 빈번하게 나오지 않는 아이템들의 집합
  
- 개념
  - S(X->Y): support(지지도). $n(X\cup Y)/N=P(X\cup Y)$
    - 전체 건수 중에서 X와 Y가 모두 포함되어 있는 건수의 비
    - 전체 집합에서 어느 정도 점유율을 차지하는지 표현
  - c(X->Y): confidence(신뢰도). $n(X\cup Y)/n(X)=P(Y|X)$
    - 항목 X를 포함하는 건수 중 X와 Y를 모두 포함하는 건수의 비
    - 어떤 규칙이 어느 정도 신뢰성이 있는지 표현
  - 향상도(Lift): $\frac{신뢰도}{P(Y)}$
    - $Lift(A,B)=\frac{c(A->B)}{s(B)}=P(Y|X)/P(Y)=P(X\cup Y)/P(X)P(Y)$
    - 그냥 B를 구매한 경우와 A를 구매한 후에 B를 구매한 경우의 차이가 클수록 큰 의미를 가짐
    - 상품 Y를 구매한 경우보다 얼마나 향상이 되었는지 보는 것
    - 1보다 크면 양의 상관, 1보다 작으면 음의 상관
    - 절대값이 1보다 클수록 좋음
  
- 규칙을 만들 때
  1. 최소 지지도를 먼저 결정 (빅데이터의 경우 지지도가 작을 수 있음)
  2. 최소 지지도보다 높은 지지도 중에서 신뢰도가 높은 규칙을 찾음
  3. 신뢰도를 높이면서 더 좋은 규칙을 찾아냄
  
- 선험 규칙: 모든 경우의 규칙패턴을 관찰하기 위해 필요한 연산 규칙
   1. 한 항목 집합이 빈발하면 그 집합의 모든 부분 집합은 빈발 항목 집합이다.
   2. 한 항목 집합이 비빈발이면 그 항목 집합을 포함하는 모든 집합은 비빈발 항목 집합이다.
   - 연산 비용을 절약 가능
   - 너무 자명해서 수학적인 증명이 필요하지 않음

In [1]:
!git clone https://github.com/joyfulspace/ADP.git

Cloning into 'ADP'...
remote: Enumerating objects: 2637, done.
remote: Counting objects: 100% (500/500), done.
remote: Compressing objects: 100% (303/303), done.
remote: Total 2637 (delta 218), reused 440 (delta 188), pack-reused 2137
Receiving objects: 100% (2637/2637), 107.64 MiB | 13.04 MiB/s, done.
Resolving deltas: 100% (222/222), done.
Checking out files: 100% (2399/2399), done.


## 실습1

In [2]:
!pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=eaa4a32e0f429c85d107e841cb8c355406c199a899438ca243b5057ef6e9f0d1
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori

In [4]:
store_data = pd.read_csv('ADP/data/store_data.csv', header=None)

In [5]:
store_data.shape

(7501, 20)

In [6]:
store_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
records = []
for i in range(0, store_data.shape[0]):
  records.append([str(store_data.values[i,j]) for j in range(0, store_data.shape[1])])

In [8]:
association_rules = apriori(records, min_support=0.0045, min_confidence=0.2, min_lift=3, min_length=2) 
                                      # 최저값에서 조금씩 높여가며 규칙 탐색.            빈발항목집합의 개수
association_results = list(association_rules)

In [9]:
len(association_results)

48

In [10]:
for item in association_results[:5]:
  pair = item[0]
  items = [x for x in pair]
  print('Rule: '+items[0]+' -> '+items[1])

  # 지지도
  print('Support: '+str(item[1]))

  # 신뢰도 & 리프트
  print('Confidence: '+str(item[2][0][2]))
  print('Lift: '+str(item[2][0][3]))
  print('===================================')

Rule: chicken -> light cream
Support: 0.004532728969470737
Confidence: 0.29059829059829057
Lift: 4.84395061728395
Rule: mushroom cream sauce -> escalope
Support: 0.005732568990801226
Confidence: 0.3006993006993007
Lift: 3.790832696715049
Rule: escalope -> pasta
Support: 0.005865884548726837
Confidence: 0.3728813559322034
Lift: 4.700811850163794
Rule: ground beef -> herb & pepper
Support: 0.015997866951073192
Confidence: 0.3234501347708895
Lift: 3.2919938411349285
Rule: ground beef -> tomato sauce
Support: 0.005332622317024397
Confidence: 0.3773584905660377
Lift: 3.840659481324083


## 실습2

In [27]:
# 엄마의 장바구니 입력
dataset = [['우유', '양파', '땅콩', '호두', '계란', '요거트'],
           ['감자', '양파', '두유', '호두', '계란', '요거트'],
           ['우유', '사과', '호두', '계란'],
           ['우유', '새우깡', '옥수수', '호두', '요거트'],
           ['옥수수', '감자', '양파', '호두', '아이스크림', '계란']]

In [28]:
# 트랜젝션 데이터로 바꿈. 장바구니 데이터를 숫자로 변경
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [29]:
df.head()

,감자,계란,두유,땅콩,사과,새우깡,아이스크림,양파,옥수수,요거트,우유,호두
0,False,True,False,True,False,False,False,True,False,True,True,True
1,True,True,True,False,False,False,False,True,False,True,False,True
2,False,True,False,False,True,False,False,False,False,False,True,True
3,False,False,False,False,False,True,False,False,True,True,True,True
4,True,True,False,False,False,False,True,True,True,False,False,True


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="ADP/file/malgun.ttf").get_name()
rc('font', family=font_name)

# top 10 시각화
top10 = pd.DataFrame(columns=['num', 'freq'])
top10['num'] = df.sum().index
top10['freq'] = df.sum().values
top10 = top10.sort_values('freq', ascending=False)
sns.barplot(data=top10, x='num', y='freq', order=top10['num'])
plt.show()

In [15]:
from mlxtend.frequent_patterns import apriori
# def apriori(df, min_support=0.5, use_colnames=False, max_len=None, n_jobs=1):
from mlxtend.frequent_patterns import association_rules

# 지지도가 0.6이상인 규칙을 추출한다.
apriori(df, min_support=0.6)

,support,itemsets
0,0.8,(1)
1,0.6,(7)
2,0.6,(9)
3,0.6,(10)
4,1.0,(11)
5,0.6,"(1, 7)"
6,0.8,"(1, 11)"
7,0.6,"(11, 7)"
8,0.6,"(9, 11)"
9,0.6,"(10, 11)"


In [16]:
apriori(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.8,(계란)
1,0.6,(양파)
2,0.6,(요거트)
3,0.6,(우유)
4,1.0,(호두)
5,0.6,"(계란, 양파)"
6,0.8,"(계란, 호두)"
7,0.6,"(양파, 호두)"
8,0.6,"(요거트, 호두)"
9,0.6,"(우유, 호두)"


In [17]:
# 아이템셋의 길이를 구하여 컬럼에 추가한다.
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.8,(계란),1
1,0.6,(양파),1
2,0.6,(요거트),1
3,0.6,(우유),1
4,1.0,(호두),1
5,0.6,"(계란, 양파)",2
6,0.8,"(계란, 호두)",2
7,0.6,"(양파, 호두)",2
8,0.6,"(요거트, 호두)",2
9,0.6,"(우유, 호두)",2


#### 최소 조합 항목 설정

In [18]:
# 파이썬 Apriori에서는 최소 신뢰도 및 조합항목 수 조절 불가
# 아래 방법으로 최소 조합 항목 설정
frequent_itemsets[ (frequent_itemsets['length']==2)&(frequent_itemsets['support']>=0.8) ]

,support,itemsets,length
6,0.8,"(계란, 호두)",2


In [19]:
frequent_itemsets[frequent_itemsets['itemsets']=={'양파', '계란'}]

,support,itemsets,length
5,0.6,"(계란, 양파)",2


In [35]:
# 신뢰도(confidence) : A,B가 동시에 일어난 횟수 / A가 일어난 횟수
# 규칙을 생성하며 최소 신뢰도 설정
rule_1 = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8)

In [36]:
# 연관규칙 생성
# 향상도(lift) : A가 주어지지 않았을 때의 B의 확률 대비 A가 주어졌을 때의 B의 확률
# 향상도가 1보다 크거나(+관계) 작다면(-관계) 우연적 기회보다 우수함
# 서로 독립이면 향상도는 1
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(계란),(양파),0.8,0.6,0.6,0.75,1.25,0.12,1.6
1,(양파),(계란),0.6,0.8,0.6,1.00,1.25,0.12,inf
2,(계란),(호두),0.8,1.0,0.8,1.00,1.00,0.00,inf
3,(호두),(계란),1.0,0.8,0.8,0.80,1.00,0.00,1.0
4,(양파),(호두),0.6,1.0,0.6,1.00,1.00,0.00,inf
5,(호두),(양파),1.0,0.6,0.6,0.60,1.00,0.00,1.0
6,(요거트),(호두),0.6,1.0,0.6,1.00,1.00,0.00,inf
7,(호두),(요거트),1.0,0.6,0.6,0.60,1.00,0.00,1.0
8,(우유),(호두),0.6,1.0,0.6,1.00,1.00,0.00,inf
9,(호두),(우유),1.0,0.6,0.6,0.60,1.00,0.00,1.0


In [37]:
rules[(rules['lift']>1)&(rules['confidence']>=0.6)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(계란),(양파),0.8,0.6,0.6,0.75,1.25,0.12,1.6
1,(양파),(계란),0.6,0.8,0.6,1.00,1.25,0.12,inf
11,"(계란, 호두)",(양파),0.8,0.6,0.6,0.75,1.25,0.12,1.6
12,"(호두, 양파)",(계란),0.6,0.8,0.6,1.00,1.25,0.12,inf
13,(계란),"(호두, 양파)",0.8,0.6,0.6,0.75,1.25,0.12,1.6
14,(양파),"(계란, 호두)",0.6,0.8,0.6,1.00,1.25,0.12,inf


In [38]:
# 규칙의 조합항목 개수 컬럼 생성
rules['antecedents_length'] = rules['antecedents'].apply(lambda x: len(x))
rules['consequents_length'] = rules['consequents'].apply(lambda x: len(x))
rules['total_length'] = rules['antecedents_length'] + rules['consequents_length']
rules = rules.sort_values('lift', ascending=False)

In [39]:
rules['antecedents_length'].value_counts()
# 좌측항이 1개 항목으로 조합된 규칙은 13건, 2개 항목으로 조합된 규칙은 3건이다.

1    13
2     3
Name: antecedents_length, dtype: int64

In [40]:
rules['consequents_length'].value_counts()
# 우측항이 1개 항목으로 조합된 규칙은 13건, 2개 항목으로 조합된 규칙은 3건이다.

1    13
2     3
Name: consequents_length, dtype: int64

In [41]:
rules['total_length'].value_counts()
# 전체 규칙 중 10개의 규칙은 2개 항목의 조합으로 이루어져 있으며 
# 6개의 규칙은 3개 항목의 조합으로 이루어져 있다.

2    10
3     6
Name: total_length, dtype: int64

In [42]:
a = rules.describe(include='all')
a

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:4012: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length,total_length
count,16,16,16.000000,16.000000,16.000000,16.000000,16.00000,16.000,16.0,16.000000,16.000000,16.000
unique,8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,(호두),(호두),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,0.775000,0.775000,0.625000,0.840625,1.09375,0.045,inf,1.187500,1.187500,2.375
std,NaN,NaN,0.177012,0.177012,0.068313,0.175327,0.12500,0.060,NaN,0.403113,0.403113,0.500
min,NaN,NaN,0.600000,0.600000,0.600000,0.600000,1.00000,0.000,1.0,1.000000,1.000000,2.000
25%,NaN,NaN,0.600000,0.600000,0.600000,0.712500,1.00000,0.000,1.0,1.000000,1.000000,2.000
50%,NaN,NaN,0.800000,0.800000,0.600000,0.900000,1.00000,0.000,NaN,1.000000,1.000000,2.000
75%,NaN,NaN,1.000000,1.000000,0.600000,1.000000,1.25000,0.120,NaN,1.000000,1.000000,3.000


In [43]:
print(a.loc['min', 'lift'])
print(a.loc['mean', 'support'])

1.0
0.625


규칙들에 대한 향상도의 최소값은 1이다. 규칙들에 대한 지지도의 평균값은 0.625이다

In [44]:
print(a.loc['min','lift'])  
### 규칙들에 대한 향상도의 최소값은 6.41로 꽤 높게 나타났음
print(a.loc['mean','support']) 
### 항목들의 교집합 확률을 의미하는 지지도의 평균은 0.625으로 나타났다.

1.0
0.625


### 특정 항목에 대한 연관성규칙 결과 확인
* frozenset: set형식과 문법, 기능이 거의 동일
* 단, set은 새로운 값의 추가 또는 삭제가 가능하지만, frozenset은 불가능

In [50]:
rules[rules['consequents']==frozenset(['양파'])]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length,total_length
0,(계란),(양파),0.8,0.6,0.6,0.75,1.25,0.12,1.6,1,1,2
11,"(계란, 호두)",(양파),0.8,0.6,0.6,0.75,1.25,0.12,1.6,2,1,3
5,(호두),(양파),1.0,0.6,0.6,0.60,1.00,0.00,1.0,1,1,2


* 총 3개의 규칙이 도출되었으며 (양파)와 (계란)이 함께 담길 확률은 지지도(support) 확인 결과 60%이다. 이 규칙의 향상도(lift)는 1.25로 (양파)만 담길 때보다 (계란)이 담기고 (양파)가 담길 확률이 약 1.25배 높다는 걸 의미한다.

In [ ]:
# Association 을 위한 전처리
import pandas as pd
from mlxtend.frequent_patterns import apriori

# 데이터 프레임 물건 리스트 변환
df = pd.DataFrame([[1, 'banana'],[2, 'banana'],[2, 'apple'],[3, 'banana']], columns=['a','b'])
def toList(x):
  return list(set(x))
df1 = df.groupby('a').b.apply(lambda x: toList(x)).reset_index()
# 2개 이상만 추출
df1['leng'] = df1.b.apply(lambda x: len(x) >= 2)

# 원하는 연관분석 형태로 변환
dataset = list(df1.b)

# 조금더 확실하게 볼수 있는 더 많은 데이터셋으로 전환
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

# 원하는 변수들을 인덱스/컬럼 으로 재정렬
# 각 제품의 포함 여부를 one-hot encoding하여 array 로 변환 
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)

# 변환된 array를 dataframe으로 변환 후 확인
df = pd.DataFrame(te_ary, columns=te.columns_)

# 컬럼 이름
print(te.columns_)

# 혹시 1 또는 0으로 변경하고 싶다면
pd.DataFrame(te_ary.astype('int'), columns=te.columns_)

# 원래 이중 리스트로 변환
te.inverse_transform(te_ary)

# 연관규칙 분석을 위한 apriori 알고리즘 사용
from mlxtend.frequent_patterns import apriori

# 지지도 도출 -> 수가 많을 수 있으므로 min_support 로 일정 이상의 지지도만 도출 (default=0.5)
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)

# 특정 개수 이상의 itemset만 추출
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[frequent_itemsets['length'] >=2]

# 특정 아이템(Eggs) 이 포함된 것만 추출
frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: 'Eggs' in list(x))]

# 연관 규칙 도출
from mlxtend.frequent_patterns import association_rules

# 최소 신뢰도 0.7이상인것만 추출
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

# 최소 향상도 0.7이상인것만 추출
association_rules(frequent_itemsets, metric="lift", min_threshold=0.7)

# antecedents 1개인거만 추출
rules[rules.apply(lambda x: True if len(x.antecedents) else false, axis=1)]

# lift 제일 큰것찾기 = 상호 정보량이 가장 큰것 찾기(lift max)
df[df.antecedents == frozenset({'Eggs'})].sort_values(by='lift', ascending=False)

# antecedents가 Eggs 이고, consequents 가 하나일때 lift 젤 작은것 찾기 = 가장 멀리있어도 되는 물품
rules[(rules.antecedents == frozenset({'Eggs'})) & (rules.consequents.apply(lambda x: len(x) ==1))].sort_values(by='lift')

# antecedents에 특정 단어 'Eggs' 있는거 찾기
rules[rules.antecedents.apply(lambda x: 'Eggs' in x)]

# 특정 antecedents 만 찾기
rules[rules.antecedents == frozenset({'Eggs'})]

# fronzenset 에서 값 추출하기
[i for i in frozenset({'Apple', 'Banana'})]